# Which key exchange method yields the smaller error

In this note book, the error for exchanging keys using three different methods is discussed. All of them realize the BB84 protocol, but the exact implementation on the quantum inspire network is varied in the spirit of this hackathon.

In [1]:
import coreapi

In [19]:
from quantuminspire.api import QuantumInspireAPI
auth = coreapi.auth.TokenAuthentication("c39a14a77f691765d2eed16742afe945ccaf572f", scheme="token")
qi = QuantumInspireAPI(authentication=auth)

In [3]:
qasm_basis = '''version 1.0

qubits 2

X q[0]
H q[0]

swap q[0],q[1]

measure_all

'''

In [58]:
def gen_qasm_circuits_2qsimple(m, b1, b2):
    for a, b, c in zip(m, b1, b2):
        string = '''version 1.0\nqubits 2'''
        if a==1: string += "\nX q[0]"
        if b==1: string += "\nH q[0]"
        string += "\nswap q[0],q[1]"
        if c==1: string += "\nH q[1]"
        string += "\nmeasure_all"
        yield string

In [5]:
def gen_qasm_circuits_2qfancy(mA, bA1, bA2, mB, bB1, bB2):
    for i in range(len(mA)):
        string = '''version 1.0\nqubits 2'''
        if mA[i]==1: string += "\nX q[0]"
        if mB[i]==1: string += "\nX q[1]"
        if bA1[i]==1: string += "\nH q[0]"
        if bB1[i]==1: string += "\nH q[1]"
        string += "\nswap q[0],q[1]"
        if bA2[i]==1: string += "\nH q[0]"
        if bB2[i]==1: string += "\nH q[1]"
        string += "\nmeasure_all"
        yield string

In [73]:
def gen_qasm_circuits_nqsimple(n, m, b1, b2):
    for i in range(len(m)):
        string = [f'version 1.0', f'qubits {n}']
        
        if m[i]==1: [string.append(f"X q[{j}]") for j in range(n)]
        if b1[i]==1: [string.append(f"H q[{j}]") for j in range(n)]
            
        if b2[i]==1: [string.append(f"H q[{j}]") for j in range(n)]
        string.append("measure_all")
        yield "\n".join(string)

def gen_qasm_circuits_2nqfancy(n, mA, bA1, bA2, mB, bB1, bB2):
    for i in range(len(mA)):
        string = [f'version 1.0', f'qubits {n}']
        if mA[i]==1: [string.append(f"X q[{j}]") for j in range(n)]
        if mB[i]==1: [string.append(f"X q[{j+n}]") for j in range(n)]
        if bA1[i]==1: [string.append(f"H q[{j}]") for j in range(n)]
        if bB1[i]==1: [string.append(f"H q[{j+n}]") for j in range(n)]
            
        [string.append(f"swap q[{j}],q[{j+n}]") for j in range(n)]
        
        if bA2[i]==1: [string.append(f"H q[{j}]") for j in range(n)]
        if bB2[i]==1: [string.append(f"H q[{j+n}]") for j in range(n)]
        string.append("measure_all")
        yield "\n".join(string)


def gen_qasm_circuits_2nqfancy_without_swap(n, mA, bA1, bA2, mB, bB1, bB2):
    for i in range(len(mA)):
        string = [f'version 1.0', f'qubits {n}']
        if mA[i]==1: [string.append(f"X q[{j}]") for j in range(n)]
        if mB[i]==1: [string.append(f"X q[{j+n}]") for j in range(n)]
        if bA1[i]==1: [string.append(f"H q[{j}]") for j in range(n)]
        if bB1[i]==1: [string.append(f"H q[{j+n}]") for j in range(n)]
        
        if bB2[i]==1: [string.append(f"H q[{j}]") for j in range(n)]
        if bA2[i]==1: [string.append(f"H q[{j+n}]") for j in range(n)]
            
        string.append("measure_all")
        yield "\n".join(string)
        
def gen_qasm_circuits_nqentagnled(n, m, b1, b2):
    assert n>1, "n needs to be bigger than 1"
    for i in range(len(m)):
        string = [f'version 1.0', f'qubits {n}']
        if m[i]==1: string.append(f"X q[0]")
        if b1[i]==1: string.append(f"H q[0]")
        [string.append(f"cnot q[0],q[{j}]") for j in range(1, n)]
        
        if b2[i]==1: [string.append(f"H q[{j}]") for j in range(n)]
        string.append("measure_all")
        yield "\n".join(string)

In [6]:
def gen_qasm_circuits_1q(m, b1, b2):
    for i in range(len(m)):
        string = '''version 1.0
qubits 1'''
        if m[i] == 1: string += "\nX q[0]"
        if b1[i] == 1: string += "\nH q[0]"
        if b2[i] == 1: string += "\nH q[0]"
        string += '''
measure_all'''
        
        yield string

In [88]:
for s in gen_qasm_circuits_2nqfancy(5, [0, 1], [1, 0], [1, 1], [0, 1], [1, 0], [1, 1]):
    print(s)
    print('----')

version 1.0
qubits 5
H q[0]
H q[1]
H q[2]
H q[3]
H q[4]
H q[5]
H q[6]
H q[7]
H q[8]
H q[9]
swap q[0],q[5]
swap q[1],q[6]
swap q[2],q[7]
swap q[3],q[8]
swap q[4],q[9]
H q[0]
H q[1]
H q[2]
H q[3]
H q[4]
H q[5]
H q[6]
H q[7]
H q[8]
H q[9]
measure_all
----
version 1.0
qubits 5
X q[0]
X q[1]
X q[2]
X q[3]
X q[4]
X q[5]
X q[6]
X q[7]
X q[8]
X q[9]
swap q[0],q[5]
swap q[1],q[6]
swap q[2],q[7]
swap q[3],q[8]
swap q[4],q[9]
H q[0]
H q[1]
H q[2]
H q[3]
H q[4]
H q[5]
H q[6]
H q[7]
H q[8]
H q[9]
measure_all
----


In [91]:
import numpy as np

In [119]:
def gen_test_inputs(N, l):
    d = np.random.randint(0, 2, (N, l, 3))
    return d[..., 0], d[..., 1], d[..., 2]

In [120]:
gen_test_inputs(10, 10)

(array([[1, 0, 1, 1, 1, 1, 1, 1, 1, 0],
        [0, 1, 1, 0, 1, 0, 0, 1, 1, 0],
        [0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
        [1, 1, 0, 0, 1, 1, 1, 1, 0, 1],
        [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 1, 1, 0, 0],
        [0, 0, 1, 1, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 1, 0, 1, 0, 1, 1, 0, 0],
        [1, 0, 0, 1, 1, 1, 0, 1, 0, 1]]),
 array([[0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 1, 0, 0, 1, 1, 0, 1, 1, 0],
        [1, 1, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
        [0, 1, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 1, 1, 0, 1, 0],
        [0, 1, 1, 1, 0, 1, 0, 1, 1, 1],
        [0, 1, 1, 1, 0, 0, 0, 1, 0, 0]]),
 array([[1, 1, 1, 1, 0, 0, 0, 1, 0, 1],
        [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
        [1, 0, 0, 1, 0, 0, 0, 0, 1, 

In [ ]:
backend_type = qi.get_backend_type_by_name('Spin-2')

np.random.seed(1337)

results = []
data = []
N, l = 10, 10
inputs = np.random.randint(0, 2, (N, l, 3))
for n in range(N):
    for cirquit in gen_qasm_circuits_2qsimple(inputs[n, :, 0], inputs[n, :, 1], inputs[n, :, 2]):
        results.append(qi.execute_qasm(cirquit, backend_type=backend_type, number_of_shots=1))

        if results[-1].get('histogram', {}):
            data.append(results[-1]['histogram'])
    #         print(results[-1]['histogram'])
        else:
            reason = results[-1].get('raw_text', 'No reason in result structure.')
            print(f'Result structure does not contain proper histogram data. {reason}')

In [125]:
len(data)

85

In [129]:

r1 = [] # 01
r2 = [] # 10
for d in data:
    keys = list(d.keys())
    maxkey = keys[0]
    for k in keys:
        if d[k] > d[maxkey]: maxkey = k
#     v = d[maxkey]
    r1.append(int(maxkey)&1)
    r2.append(int((int(maxkey)&2)/2))
    

In [ ]:
# calculate number of errors:
n_errors = np.sum(np.abs(k1-k2), axis=-1)

In [86]:
results

[{'id': 7111615,
  'url': 'https://api.quantum-inspire.com/results/7111615/',
  'job': 'https://api.quantum-inspire.com/jobs/7124522/',
  'created_at': '2022-01-30T11:20:02.090386Z',
  'number_of_qubits': 2,
  'execution_time_in_seconds': 0.0030265,
  'raw_text': '',
  'raw_data_url': 'https://api.quantum-inspire.com/results/7111615/raw-data/4f0c4eee218d4487c8f1ee5f52d4adba147bf6999935f8963c92100ff1bfedf6/',
  'histogram': OrderedDict([('2', 1.0)]),
  'histogram_url': 'https://api.quantum-inspire.com/results/7111615/histogram/4f0c4eee218d4487c8f1ee5f52d4adba147bf6999935f8963c92100ff1bfedf6/',
  'measurement_mask': 3,
  'quantum_states_url': 'https://api.quantum-inspire.com/results/7111615/quantum-states/4f0c4eee218d4487c8f1ee5f52d4adba147bf6999935f8963c92100ff1bfedf6/',
  'measurement_register_url': 'https://api.quantum-inspire.com/results/7111615/measurement-register/4f0c4eee218d4487c8f1ee5f52d4adba147bf6999935f8963c92100ff1bfedf6/',
  'calibration': 'https://api.quantum-inspire.com/c

In [81]:
results[0]

{'id': 7111615,
 'url': 'https://api.quantum-inspire.com/results/7111615/',
 'job': 'https://api.quantum-inspire.com/jobs/7124522/',
 'created_at': '2022-01-30T11:20:02.090386Z',
 'number_of_qubits': 2,
 'execution_time_in_seconds': 0.0030265,
 'raw_text': '',
 'raw_data_url': 'https://api.quantum-inspire.com/results/7111615/raw-data/4f0c4eee218d4487c8f1ee5f52d4adba147bf6999935f8963c92100ff1bfedf6/',
 'histogram': OrderedDict([('2', 1.0)]),
 'histogram_url': 'https://api.quantum-inspire.com/results/7111615/histogram/4f0c4eee218d4487c8f1ee5f52d4adba147bf6999935f8963c92100ff1bfedf6/',
 'measurement_mask': 3,
 'quantum_states_url': 'https://api.quantum-inspire.com/results/7111615/quantum-states/4f0c4eee218d4487c8f1ee5f52d4adba147bf6999935f8963c92100ff1bfedf6/',
 'measurement_register_url': 'https://api.quantum-inspire.com/results/7111615/measurement-register/4f0c4eee218d4487c8f1ee5f52d4adba147bf6999935f8963c92100ff1bfedf6/',
 'calibration': 'https://api.quantum-inspire.com/calibration/109

In [48]:
client = coreapi.Client(auth=auth)

In [85]:
client.get(
f'https://api.quantum-inspire.com/results/7111615/histogram/4f0c4eee218d4487c8f1ee5f52d4adba147bf6999935f8963c92100ff1bfedf6/')

ErrorMessage: <Error: 404 Not Found>
    detail: "Not found."

In [50]:
client.get(f'{results[0]["calibration"]}')

OrderedDict([('url', 'https://api.quantum-inspire.com/calibration/109108/'),
             ('backend', 'https://api.quantum-inspire.com/backends/7/'),
             ('parameters',
              OrderedDict([('system',
                            OrderedDict([('fridge_temperature',
                                          OrderedDict([('value', 0.0207),
                                                       ('symbol',
                                                        'Fridge temperature'),
                                                       ('unit', 'K'),
                                                       ('decimals', 2)])),
                                         ('last_calibration_date',
                                          OrderedDict([('value',
                                                        '2022-01-27T11:23:19.435562'),
                                                       ('symbol',
                                                        'Calibration da

In [36]:
client.get(f'https://api.quantum-inspire.com/results/7107766/quantum-states/54d8d2b26681e09cc12b355b4964b4dd8ece2a0fee26a113ce0591715feecacb')

ErrorMessage: <Error: 404 Not Found>
    detail: "Not found."